# Serie 6
## Esercizio 1 
### Problema
Il problema (ID inconsistente) è causato dal `this` inserito nel costruttore di `EventListener`. La referenza `this` infatti non è valida nel caso in cui l'oggetto viene creato per metà (context switch).  

```java
public EventListener(final int id, final EventSource eventSource) {
		eventSource.registerListener(id, this); 
        // Il `this` in questo punto potrebbe riferirsi ad 
        // un oggetto in uno stato inconsistente (!)
		try {
			Thread.sleep(4);
		} catch (final InterruptedException e) {}
		this.id = id;
	}
```

### Risoluzione
Per risolvere il problema è possibile effettuare la registrazione dell'evento una volta inizializzato l'oggetto (in questo modo siamo sicuri che il costruttore ha completato il suo lavoro in modo corretto).

```java
class EventListener {
	private final int id;

	public EventListener(final int id, final EventSource eventSource) {
		try {
			Thread.sleep(4);
		} catch (final InterruptedException e) {}
		this.id = id;
	}

	public void register(EventSource es){
		if(es != null){
			es.registerListener(id, this);
		}
	}

    // ...
}
```

Possiamo ora inizializzare gli `EventListener` nel for-loop e registrarli alla sorgente di eventi una volta che questi sono stati istanziati (e che quindi si trovano in uno stato consistente:
```java
final List<EventListener> allListeners = new ArrayList<>();
for (int i = 1; i <= 20; i++) {
    EventListener el = new EventListener(i, eventSource);
    el.register(eventSource);
    allListeners.add(el);
}
```

## Esercizio 2
### Problema
Innanzitutto si verifica un problema di visibilità, in quanto `sharedState` non è volatile, di conseguenza il suo stato è differente fra i thread.  
Inoltre, si presenta un problema race condition con l'istanza di SharedState che esegue un operazione di incremento in modo non atomico.  

### Risoluzione
Dobbiamo innanzitutto rendere `sharedState` volatile:
```java
static volatile IState sharedState = null;
```
successivamente dobbiamo sistemare la race condition. Per farlo senza toccare le classi `SharedState` e `ThreadSafeSharedState` possiamo usare un `synchronized` sulle chiamate alle funzioni di `increment` e `getValue`, ma questo comporterrebbe un overhead inutile in caso si usi `ThreadSafeSharedState`.  
Per ovviare al problema possiamo usare un Holder in modo da rendere l'oggetto thread safe:
```java
// Choose which share to instantiate
if (S6Esercizio2.THREADSAFE_SHARE) {
    S6Esercizio2.sharedState = new ThreadSafeSharedState();
} else {
    S6Esercizio2.sharedState = new Synchronize(new SharedState());
}
```

## Holder degli oggetti Immutable
Una soluzione però più efficiente risulta essere quella di utilizzare un holder di oggetti immutable, ossia avere una classe `Holder` che, ad ogni cambiamento di variabile ritorni una nuova istanza di se stessa. In questo modo possiamo sapere per certo che se l'indirizzo puntato è cambiato anche il suo stato è variato.  
```java
final class ImmutableSharedState implements IState {

	private int value = 0;

	public ImmutableSharedState(int value){
		this.value = value;
	}

	@Override
	public void increment() {
        // Should never be called!
	}

	@Override
	public int getValue() {
		return value;
	}
}
```

Possiamo quindi trasformare l'istanziazione ed il for-loop come segue:
```java
S6Esercizio2_HIO.sharedState = new ImmutableSharedState();

// Sleep before updating
try {
    Thread.sleep(1000);
} catch (final InterruptedException e) {
}

// Perform 5000 increments and exit
System.out.println("Starter : begin incrementing");
for (int i = 0; i < 5000; i++) {
    S6Esercizio2_HIO.sharedState = new ImmutableSharedState(
        S6Esercizio2_HIO.sharedState.getValue() + 1
    );
    if ((i % 100) == 0) {
        try {
            Thread.sleep(1);
        } catch (final InterruptedException e) { }
    }
}
System.out.println("Starter : completed");
```

## Esercizio 3
Personalmente avrei reso `ExchangeRates` immutable, però mi sono reso conto che nella serie precedente avevo già utilizzato questa tecnica.  
Ritengo dunque che il codice non sia migliorabile ulteriormente.

```java
public volatile ExchangeRates currentER;

private ExchangeRates generateRandomER() {
    double[] er = new double[ExchangeRates.size];
    for(int i = 0; i<ExchangeRates.size; i++){
        er[i] = Math.random() + 0.5;
    }
    return new ExchangeRates(er);
}

public void getNewExchangeRates() {
    System.out.println("Getting new exchange rates!");
    readWriteLock.readLock().lock();
    currentER = generateRandomER();
    System.out.println("New ER: " + currentER);
    readWriteLock.readLock().unlock();
}
```

## Esercizio 4

Il problema è causato da `synchronized` su `occupaBagno(final boolean uomo)` che accoda i 45 ms di `Thread.sleep` alla chiamata del metodo.  
Per risolvere il problema è sufficiente piazzare i blocchi synchronized nel modo corretto. Inoltre, per migliorare le performances è possibile usare uno stack in modo da evitare inutili controlli sui bagni.  
  
```java
private final Stack<Bagno> bagniUomini;
private final Stack<Bagno> bagniDonne;
// ...
public boolean occupaBagno(final boolean uomo) {
    Bagno bagnoOccupato = null;

    synchronized (this) {
        if (uomo) {
            // Cerca primo bagno libero per uomini
            if (bagniUomini.empty()) {
                return false;
            }

            Bagno bagno = bagniUomini.peek();
            while (!bagniUomini.empty() && !bagno.provaOccupare()) {
                bagno = bagniUomini.pop();
            }

            bagnoOccupato = bagno;
        } else {
            // Same for women
            // ...
        }
   }
   
   // tutti i bagni sono occupati!
    if (bagnoOccupato == null) {
        return false;
    }

    try {
        Thread.sleep(45);
    } catch (final InterruptedException e) {
        e.printStackTrace();
    }

    synchronized (this) {
        // Libera bagno
        bagnoOccupato.libera();
        if (uomo) {
            bagniUomini.push(bagnoOccupato);
        } else {
            bagniDonne.push(bagnoOccupato);
        }
    }

    return true;
}
```